<a href="https://colab.research.google.com/github/Redwoods/Py/blob/master/pdm2020/my-note/py-tensorflow/DL1-FCN/mnist06_FCN_test_ALL_best_model_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DNN (Deep Neural Networks) 구현 방법
> - **Sequential model**
> - Functional model
> - Subclassing model

# **Test of mnist FCN model**
## Sequential model
- FCN3

***

![mnist_NN_2_layers](https://raw.githubusercontent.com/Redwoods/Py/master/pdm2020/my-note/py-tensorflow/images/mnist_NN_2_layers.png)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline

print(tf.__version__)

## **데이터 로딩, 정규화**

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train/255.0, X_test/255.0 # 정규화

print("X_train={0}".format(X_train.shape))
print("y_train={0}".format(y_train.shape)) 
print("X_test={0}".format(X_test.shape))
print("y_test={0}".format(y_test.shape))

In [ ]:
# One-Hot-Encoding
# Use function to_categorical() to do One-Hot-Encoding
# tf.keras.utils.to_categorical
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## **FCN3**

### Control of training process
- EarlyStopping
    - patience
- ModelCheckpoint
    > Save best model during training

In [ ]:
from tensorflow.keras import callbacks
# from callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
mc_callback = callbacks.ModelCheckpoint(filepath="./mnist_fcn3_best_weights.{epoch:03d}-{val_accuracy:.3f}.hdf5", 
                              monitor='val_accuracy', verbose=0, save_best_only=True)
es_callback = callbacks.EarlyStopping(monitor='val_accuracy', 
                            mode='max', verbose=1, patience=20) 

In [ ]:
# set random number seed
np.random.seed(1234)
tf.random.set_seed(1234)

fcn3 = tf.keras.models.Sequential([ 
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),  
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation='relu'),  
    tf.keras.layers.Dropout(0.25), 
    tf.keras.layers.Dense(10, activation='softmax')
])

fcn3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']) 


In [ ]:
fcn3.summary()

In [ ]:
# Training with callbacks
hist = fcn3.fit(X_train,y_train, 
                epochs= 100 , 
                batch_size =100,
                validation_data=(X_test,y_test),
                callbacks=[mc_callback, es_callback])

In [ ]:
fcn3.evaluate(X_test, y_test, batch_size= 100)

In [ ]:
# 훈련 데이터와 검증 데이터에 대한 loss 시각화.
epochs = range(1, len(hist.history['loss']) + 1)

loss_list = hist.history['loss'] #[100 * i for i in history.history['loss']]
vloss_list = hist.history['val_loss'] #[100 * i for i in history.history['val_loss']]

plt.plot(epochs,loss_list)  
plt.plot(epochs,vloss_list)

plt.plot(np.argmin(np.array(vloss_list))+1,vloss_list[np.argmin(np.array(vloss_list))], 'r*')
plt.title('mnist: NN(FCN) model - val_loss, min:' + str(np.round(vloss_list[np.argmin(np.array(vloss_list))],2)))
plt.ylabel('val-Loss (%)')
plt.xlabel('Epoch')
plt.legend(['loss','val_loss','best'], loc='best')
plt.show()

In [ ]:
# 훈련 데이터와 검증 데이터에 대한 accuracy 시각화.
epochs = range(1, len(hist.history['accuracy']) + 1)

acc_list = [100 * i for i in hist.history['accuracy']]
vacc_list = [100 * i for i in hist.history['val_accuracy']]

plt.plot(epochs,acc_list)  
plt.plot(epochs,vacc_list)

plt.plot(np.argmax(np.array(vacc_list))+1,vacc_list[np.argmax(np.array(vacc_list))], 'r*')
plt.title('mnist: NN(FCN) model - val_accuracy, max:' + str(np.round(vacc_list[np.argmax(np.array(vacc_list))],2)))
plt.ylabel('val-Accuracy (%)')
plt.xlabel('Epoch')
plt.legend(['accuracy','val_accuracy','best'], loc='best')
plt.show()

## 모형의 재사용

- 트레이닝 중 hdf5 형식으로 best model이 저장됨.
- load 명령으로 불러 사용
> best model: mnist_fcn3_best_weights.020-0.986.hdf5 

>> mnist_fcn3_best_weights.044-0.985.hdf5

In [ ]:
from keras.models import load_model

best_model = load_model('mnist_fcn3_best_weights.044-0.985.hdf5')
best_model.evaluate(X_test, y_test, batch_size=100)

---

## Visualization of the best model
---
![mnist_fcn_best](https://raw.githubusercontent.com/Redwoods/Py/master/pdm2020/my-note/py-streamlit-21/app/mnist_fcn.png)

In [ ]:
def plot_pred(i, pred_y, true_y, img):
    
    plt.grid(False)
    plt.title(class_names[np.argmax(true_y)])
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img)

    predicted_label = np.argmax(pred_y)
    if predicted_label == np.argmax(true_y):   # np.argmax(true_label)
        plt.suptitle("Test image-%d: Classified correctly" % i)
    else:
        plt.suptitle("Test image-%d: Wrong classification" % i)


def plot_bar(pred_y, true_y):
    
    plt.grid(False)
    plt.yticks([])
    plt.xticks(np.arange(10), class_names, rotation=0)

    thisplot = plt.bar(range(10), pred_y, color='grey')
    plt.ylim([0, 1])
    predicted_label = np.argmax(pred_y)

    if predicted_label == np.argmax(true_y):  
        color = 'green'
    else:
        color = 'red'

    thisplot[predicted_label].set_color(color)


In [ ]:
X_test.shape,y_test.shape

In [ ]:
num = np.random.randint(0, len(y_test))
y_pred0 = best_model.predict(X_test[num:num+1], verbose=1)

In [ ]:
y_pred0

In [ ]:
y_test0=y_test[num:num+1]
y_test0

In [ ]:
X_test0 = X_test[num]

In [ ]:
class_names = ["0","1","2","3","4","5","6","7","8","9"]

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plot_pred(num, y_pred0[0], y_test0[0], X_test0)
plt.subplot(2,1,2)
plot_bar(y_pred0[0],  y_test0[0])
plt.title("Green: True, Red: False")
plt.show()

---

In [ ]:
# Random test
num = np.random.randint(0, len(y_test))
y_pred0 = best_model.predict(X_test[num:num+1], verbose=1)
y_test0=y_test[num:num+1]
X_test0 = X_test[num]

class_names = ["0","1","2","3","4","5","6","7","8","9"]

plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plot_pred(num, y_pred0[0], y_test0[0], X_test0)
plt.subplot(2,1,2)
plot_bar(y_pred0[0],  y_test0[0])
plt.title("Green: True, Red: False")
plt.show()